In [11]:
# Install required Python packages
!pip install --quiet flask pyngrok pytesseract googletrans==4.0.0-rc1 opencv-python-headless

# Install language models for French and Arabic (may need root permission workaround)
!sudo apt-get update > /dev/null
!sudo apt-get install -y tesseract-ocr-fra tesseract-ocr-ara > /dev/null

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
import pytesseract
import numpy as np
import cv2
from googletrans import Translator
import io
from pyngrok import ngrok, conf

conf.get_default().auth_token = "2jsd7gMQ3m9uuda9evKjJPGvQ3l_3yWj2KrEXvcJWN1JT5bHA"
app = Flask(__name__)

@app.route("/process", methods=["POST"])
def process_image():
    if "image" not in request.files:
        return jsonify({"error": "No image uploaded"}), 400

    file = request.files["image"]
    image = Image.open(io.BytesIO(file.read())).convert("RGB")
    image_np = np.array(image)
    image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)


    langs = ['fra', 'eng', 'ara']
    extracted_text = ""
    used_lang = ""
    for lang in langs:
        try:
            text = pytesseract.image_to_string(image_cv, lang=lang).strip()
            if text:
                extracted_text = text
                used_lang = lang
                break
        except:
            continue

    if not extracted_text:
        return jsonify({"error": "Text could not be extracted"}), 400

    # Translate
    translator = Translator()
    translations = {}
    targets = {"English": "en", "French": "fr", "Arabic": "ar"}

    for name, code in targets.items():
        try:
            translations[name] = translator.translate(extracted_text, dest=code).text
        except:
            translations[name] = "Translation failed."

    # Simulate Tunisian dialect (approximation from Arabic)
    tunisian = ""
    try:
        arabic = translator.translate(extracted_text, dest="ar").text
        tunisian = arabic  # Optionally apply rule-based or slang here
    except:
        tunisian = "Dialect translation failed."

    return jsonify({
        "extracted": extracted_text,
        "used_lang": used_lang,
        "translations": translations,
        "tunisian": tunisian
    })

public_url = ngrok.connect(5000)
print("Public URL (copy this to app.py):", public_url)
app.run(port=5000)

Public URL (copy this to app.py): NgrokTunnel: "https://5a66-35-192-211-198.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Jun/2025 16:48:46] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jun/2025 17:14:24] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jun/2025 17:27:01] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jun/2025 17:48:04] "POST /process HTTP/1.1" 200 -
